# Faster R-CNN Implementation

##Toolbox

In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import doctest
import os
import random

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.layers import Flatten, Dense, Input, Conv2D, MaxPooling2D, Dropout, Layer, Concatenate
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, TimeDistributed

seed = 1111

In [0]:
doctest.testmod(verbose=True)
def iou(bbox1, bbox2):
  '''
  Bbox format must be [x_min,y_min,x_max,y_max]
  >>> iou([10,10,10,10],[5,5,5,5])
  0
  >>> iou([0,0,4,4],[2,2,4,4])
  0.25
  >>> iou([0,0,4,4],[2,2,6,6])
  0.14285714285714285
  '''

  xmin_inter = max(bbox1[0],bbox2[0])
  ymin_inter = max(bbox1[1],bbox2[1])
  xmax_inter = min(bbox1[2],bbox2[2])
  ymax_inter = min(bbox1[3],bbox2[3])

  width_inter = max(xmax_inter - xmin_inter,0)
  height_inter = max(ymax_inter - ymin_inter,0)
  if(width_inter == 0 or height_inter == 0):
    iou = 0
  else:
    iou = width_inter*height_inter/((bbox1[2]-bbox1[0])*((bbox1[3]-bbox1[1]))+(bbox2[2]-bbox2[0])*((bbox2[3]-bbox2[1]))-width_inter*height_inter)
  return iou


1 items had no tests:
    __main__
0 tests in 1 items.
0 passed and 0 failed.
Test passed.


In [0]:
def calculate_delta(bbox1,bbox2):
  '''
  Bbox format must be [x_min,y_min,x_max,y_max]
  return the delta between ground bbox (bbox1) and anchors(bbox2)
  '''
  xmin_d = bbox1[0] - bbox2[0]
  ymin_d = bbox1[1] - bbox2[1]
  xmax_d = bbox1[2] - bbox2[2]
  ymax_d = bbox1[3] - bbox2[3]
  return xmin_d,ymin_d,xmax_d,ymax_d

##Architecture

##Input

In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
base_filepath = 'My Drive/SoccerAI/train_resources'
# Input file for training is train_annotation.txt
train_filepath = base_filepath+'/train_annotation.txt'
train_df = pd.read_csv(train_filepath,header=None,names=['ID','Filepath','XMin','YMin','XMax','YMax','Class'])
train_df = train_df.set_index('ID')
train_imgs = train_df.index.unique()

In [0]:
classes = train_df['Class'].unique()

def class_dict(labels):
  '''
  Create a dictionnary from the list of available classes
  Output : {1:Class1, ....n:Classn, n+1:Background}
  '''
  class_dict = {}
  j = 0
  for label in labels:
    class_dict.update({j:label})
    j+=1
  class_dict.update({j:'bg'})
  revert_class_dict = {}
  for k,v in class_dict.items():
    revert_class_dict.update({v:k})
  return class_dict,revert_class_dict
labels_dict,labels_revert_dict = class_dict(classes)

In [0]:
def input_train_gen(df,labels_revert_dict):
  img_ids = df['ID'].unique()
  for img_id in img_ids:
    bboxes = []
    sub_df = df[df['ID']==img_id]
    for index,row in sub_df:
      class_id = labels_revert_dict.get(row['Class'])
      xmin = row['XMin']
      ymin = row['YMin']
      xmax = row['XMax']
      ymax = row['YMax']
      bboxes.append([xmin,ymin,xmax,ymax,class_id])
    yield bboxes

In [0]:
img_height = 600
img_width = 800
img_depth = 3
input_shape = (img_height,img_width,img_depth)

def input_tensor(input_shape):
  x = Input(shape = input_shape)
  return x

## Base Model

In [0]:
vgg_ratio = 16

In [0]:
def vgg16(input_tensor):
      
    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(input_tensor)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)

    return x

In [0]:
def get_weights(weights):
  return None

##RPN

In [0]:
rpn_height = img_height/vgg_ratio
rpn_width = img_width/vgg_ratio
rpn_depth = 512
anchors_ratios = [0.5,1,1.5]
anchors_size = [64,128,256]
anchors_count = len(anchors_ratios) * len(anchors_size)


def rpn(x):
  
  # Mutual Layer
  x1 = Conv2D(512,(3,3), activation='relu', padding='same', kernel_initializer='normal', name='rpn_mutual_layer')(x)
  
  # Classification Layer
  x_class = Conv2D(anchors_count, (1,1), activation = 'sigmoid', kernel_initializer = 'uniform', name='rpn_class_layer')(x)
  
  # Regression Layer
  x_reg = Conv2D(4*anchors_count, (1,1), activation='linear', kernel_initializer='zero', name='rpn_reg_layer')(x)
  return [x_class,x_reg]

In [0]:
# Anchors Rep shape is (rpn_height,rpn_width,len(anchors_ratio)*len(anchors),4) stands for xmin,ymin,xmax,ymax
def anchors_list(vgg_ratio,rpn_height,rpn_width,anchors_ratios,anchors_size):
  anchors = np.zeros((rpn_height,rpn_width,len(anchors_ratios)*len(anchors_size),4))
  for i in range(0,rpn_height):
    for j in range(0,rpn_width):
      k = 0
      for ratio in anchors_ratios:
        for size in anchors_size:
          xcenter = i*vgg_ratio + vgg_ratio/2
          ycenter = j*vgg_ratio + vgg_ratio/2
          anchors_width = ratio*size
          anchors_height = size
          xmin = xcenter - anchors_width/2
          ymin = ycenter - anchors_height/2
          xmax = xcenter + anchors_width/2
          ymax = ycenter + anchors_height/2
          anchors[i][j][k][0] = xmin
          anchors[i][j][k][1] = ymin
          anchors[i][j][k][2] = xmax
          anchors[i][j][k][3] = ymax
          k+=1
  return anchors

anchors = anchors_list(vgg_ratio,int(rpn_height),int(rpn_width),anchors_ratios,anchors_size)

In [0]:
bg_threshold = 0.1
fg_threshold = 0.5
batch_size_rpn = 256
batch_size_cnn = 64
# Sampling the anchors to have balanced batch
def create_sample_rpn(pos,neut,neg,batch_size):
  '''
  Create a balanced minibatch from positive anchors, Negative anchors and complete with neutral anchors if required
  '''
  half_batch = int(batch_size/2)
  sample = []
  num_pos = len(pos)
  num_neg = len(neg)
  random.shuffle(pos)
  neut = sorted(neut, key=lambda x: x[1], reverse=True)
  random.shuffle(neg)
  pos = [x[0] for x in pos]
  neg = [x[0]for x in neg]
  if(num_pos >= half_batch):
    pos = pos[:half_batch]
  else:
    neut_temp = [x[0] for x in neut[:(half_batch-num_pos)]]
    pos = pos + neut_temp
  if(num_neg >= half_batch):
    neg = neg[:half_batch]
  else:
    neut_temp = [x[0] for x in neut[-(half_batch-num_neg):]]
    neg = neg + neut_temp
  assert len(pos) == int(batch_size/2)
  assert len(neg) == int(batch_size/2)
  return pos,neg

def create_sample_cnn(pos,neut,neg,batch_size):
  '''
  Create a balanced minibatch from positive anchors, Negative anchors and complete with neutral anchors if required
  '''
  pos_batch = int(batch_size/4)
  neg_batch = batch_size - pos_batch
  sample = []
  num_pos = len(pos)
  num_neg = len(neg)
  bg = neg[0][1]
  random.shuffle(pos)
  neut = sorted(neut, key=lambda x: x[1], reverse=True)
  random.shuffle(neg)
  if(num_pos >= pos_batch):
    pos = pos[:pos_batch]
  else:
    neut_temp = [x[0,3] for x in neut[:(half_batch-num_pos)]]
    neut = neut[len(neut)-(pos_batch-num_pos):]
    pos = pos + neut_temp
  if(len(neut) >= neg_batch):
    random.shuffle(neut)
    neg =[[x[0],bg] for x in neut[:neg_batch]]
    neg = neg[:neg_batch]
  else:
    neut_temp = [[x[0],bg] for x in neut]
    neg = neut_temp + neg[:neg_batch-len(neut_temp)]
  assert len(pos) == pos_batch
  assert len(neg) == neg_batch
  return pos,neg

# Calculation of IoU for each anchors
def anchors_iou(anchors,ground_truth_bboxes,bg_threshold,fg_threshold, batch_size_rpn,batch_size_cnn,labels_revert_dict):
  '''
  anchors input parameter is the lsit of anchors returned by anchors_list()
  ground_truth_bboxes is the list of ground truth bboxes for a giving image. Shape is (?,5) with 5 stands for xmin,ymin,xmax,ymax,class
  Output1 : anchors_cls_rpn which shape is (?,rpn_width,rpn_height,num_anchors*2). 
  Last dim from 0 to num_anchors indicates whether the anchors is in the training batch. From num_anchors to 2*num_anchors indicates
  the target value
  Output2 : anchors_reg_rpn which shape is (?,rpn_width,rpn_height,num_anchors*4*2)
  Output3 : anchors_cls_cnn which shape is (?,rpn_width,rpn_height,num_anchors*k*2).
  Output4 : anchors_reg_cnn which shape is (?,rpn_width,rpn_height,num_anchors*4*k*2)

  '''
  assert fg_threshold > bg_threshold
  neg = []
  neut = []
  pos = []
  class_count = len(labels_revert_dict)
  anchors_reg_rpn = np.zeros((len(anchors),len(anchors[0]),anchors_count*2*4))
  anchors_cls_rpn = np.zeros((len(anchors),len(anchors[0]),anchors_count*2))
  anchors_reg_cnn = np.zeros((len(anchors),len(anchors[0]),anchors_count*2*4*class_count))
  anchors_cls_cnn = np.zeros((len(anchors),len(anchors[0]),anchors_count*2*class_count))
  for i in range(0,len(anchors)):
    for j in range(0,len(anchors[i])):
      for k in range(0,len(anchors[i][j])):
        iou_temp = 0
        for bbox in ground_truth_bboxes:
          if(iou_temp < iou(bbox,anchors[i][j][k])):
            iou_temp = iou(bbox,anchors[i][j][k])
            dxmin,dymin,dxmax,dymax = calculate_delta(bbox[:4],anchors[i][j][k])
            class_id = bbox[4]
            anchors_reg_rpn[i][j][k*4+4*anchors_count] = dxmin
            anchors_reg_rpn[i][j][k*4+1+4*anchors_count] = dymin
            anchors_reg_rpn[i][j][k*4+2+4*anchors_count] = dxmax
            anchors_reg_rpn[i][j][k*4+3+4*anchors_count] = dymax
            anchors_reg_cnn[i][j][class_id*4*k+4*anchors_count*class_count] = dxmin
            anchors_reg_cnn[i][j][class_id*4*k+1+4*anchors_count*class_count] = dymin
            anchors_reg_cnn[i][j][class_id*4*k+2+4*anchors_count*class_count] = dxmax
            anchors_reg_cnn[i][j][class_id*4*k+3+4*anchors_count*class_count] = dymax
        if(iou_temp > fg_threshold):
          pos.append([(i,j,k),class_id])
        elif(iou_temp < bg_threshold):
          class_id = labels_revert_dict.get('bg')
          neg.append([(i,j,k),class_id])
        else:
          neut.append([(i,j,k),iou_temp,class_id])
  pos_rpn,neg_rpn = create_sample_rpn(pos,neut,neg,batch_size_rpn)
  pos_cnn,neg_cnn = create_sample_cnn(pos,neut,neg,batch_size_cnn)
  
  for indexes in pos_rpn:
    anchors_cls_rpn[indexes[0]][indexes[1]][indexes[2]]=1
    anchors_cls_rpn[indexes[0]][indexes[1]][indexes[2] + anchors_count]=1
    anchors_reg_rpn[indexes[0]][indexes[1]][indexes[2]*4]=1
    anchors_reg_rpn[indexes[0]][indexes[1]][indexes[2]*4 + 1]=1
    anchors_reg_rpn[indexes[0]][indexes[1]][indexes[2]*4 + 2]=1
    anchors_reg_rpn[indexes[0]][indexes[1]][indexes[2]*4 + 3]=1
  for indexes in neg_rpn:
    anchors_cls_rpn[indexes[0]][indexes[1]][indexes[2]]=1
  for indexes in pos_cnn:
    index = indexes[0]
    class_num = indexes[1]
    anchors_cls_cnn[index[0]][index[1]][index[2]*class_num]=1
    anchors_cls_cnn[index[0]][index[1]][index[2]*class_num + anchors_count*class_count]=1
    anchors_reg_cnn[index[0]][index[1]][index[2]*class_num*4]=1
    anchors_reg_cnn[index[0]][index[1]][index[2]*class_num*4 + 1]=1
    anchors_reg_cnn[index[0]][index[1]][index[2]*class_num*4 + 2]=1
    anchors_reg_cnn[index[0]][index[1]][index[2]*class_num*4 + 3]=1
  for indexes in neg_cnn:
    index = indexes[0]
    class_num = indexes[1]
    anchors_cls_rpn[index[0]][index[1]][index[2]*class_num]=1
  return anchors_cls_rpn,anchors_reg_rpn,anchors_cls_cnn,anchors_reg_cnn

y_cls_rpn,y_reg_rpn,y_cls_cnn,y_reg_cnn = anchors_iou(anchors,[[0.5*img_width,0.5*img_height,0.8*img_width,0.8*img_height,1]],
                                      bg_threshold,fg_threshold,batch_size_rpn,batch_size_cnn,labels_revert_dict)

print(y_cls_rpn.shape)
print(y_reg_rpn.shape)
print(y_cls_cnn.shape)
print(y_reg_cnn.shape)

(37, 50, 18)
(37, 50, 72)
(37, 50, 54)
(37, 50, 216)


In [0]:
def rpn_binary_crossentropy(anchors_count):
  '''
  The shape of y_pred is (rpn_height,rpn_width,anchors_count)
  The shape of y_true is (rpn_height,rpn_width,anchors_count*2) [:,:,:anchors_count] is validity tensor and [:,:,anchors_count:] is GT
  '''
  def rpn_binary_crossentropy_fixed_anchors(y_true,y_pred):

    return 1 * K.sum(y_true[:,:,:anchors_count] * K.binary_crossentropy(y_true[:,:,anchors_count:]))/K.sum(y_true[:,:,:anchors_count])
  return rpn_binary_crossentropy_fixed_anchors

HUBER_DELTA = 0.5
def rpn_smooth_l1(anchors_count):
  '''
  The shape of y_pred is (rpn_height,rpn_width,anchors_count*4)
  The shape of y_true is (rpn_height,rpn_width,anchors_count*2*4) [:,:,:anchors_count*4] is validity tensor and [:,:,anchors_count*4:] is GT
  '''
  def rpn_smooth_l1_fixed_anchors(y_true,y_pred):

    x   = K.abs(y_true[:,:,anchors_count*4:] - y_pred)
    x   = K.switch(x < HUBER_DELTA, 0.5 * x ** 2, HUBER_DELTA * (x - 0.5 * HUBER_DELTA))
    return K.sum(y_true[:,:,:anchors_count*4]*x)/K.sum(y_true[:,:,:anchors_count*4])
  return rpn_smooth_l1_fixed_anchors

##ROI

In [0]:
class ROIPoolingLayer(Layer):
    '''
    Input will be : [VGG16 Feature Layers, Proposal]
    Shape is [(1,rpn_width,rpn_height,512),(1,x,y,h,w)]
    '''
    def __init__(self, pooling_size, **kwargs):
        
        self.pooling_size = pooling_size
        
        super(ROIPoolingLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.nb_channels = input_shape[0][3]

    def call(self, x):
        assert len(x) == 2
        img = x[0]
        roi = x[1]
        
        x = roi[0,0]
        y = roi[0,1]
        h = roi[0,2]
        w = roi[0,3]
        
        x = K.cast(x, 'int32')
        y = K.cast(y, 'int32')
        w = K.cast(w, 'int32')
        h = K.cast(h, 'int32')


        output = tf.image.resize(img[:, y:y+h, x:x+w, :], (self.pooling_size, self.pooling_size))
        output = K.reshape(output , (1, self.pooling_size, self.pooling_size, self.nb_channels))
        return output

    def compute_output_shape(self, input_shape):
        return None, self.pooling_size, self.pooling_size, self.nb_channels

##R-CNN

In [0]:
classes_count = 3
def rcnn(x):
  
  # Flatten Layer
  x = Flatten()(x)
  
  # 1st Dense Layer
  x = Dense(4096,activation='relu',kernel_initializer='normal', name='rcnn_dense1' )(x)

  # 2nd Dense Layer
  x = Dense(4096,activation='relu',kernel_initializer='normal', name='rcnn_dense2')(x)
  
  # Classification Layer
  x_class = Dense(classes_count+1, activation = 'softmax', kernel_initializer = 'uniform', name='rcnn_class_layer')(x)
  
  # Regression Layer
  x_reg = Dense(4*classes_count, activation='linear', kernel_initializer='zero', name='rcnn_reg_layer')(x)
  
  return [x_class,x_reg]

In [0]:
def rcnn_multiclass_crossentropy(anchors_count,num_class):
  '''
  The shape of y_pred is (rpn_height,rpn_width,anchors_count*num_class)
  The shape of y_true is (rpn_height,rpn_width,anchors_count*2*num_class) [:,:,:anchors_count*num_class] is validity tensor and [:,:,anchors_count*num_class:] is GT
  '''
  def rcnn_multiclass_crossentropy_fixed_anchors(y_true,y_pred):

    return 1 * K.sum(y_true[:,:,:anchors_count*num_class] * K.categorical_crossentropy(y_true[:,:,anchors_count*num_class:]))/K.sum(y_true[:,:,:anchors_count*num_class])
  return rcnn_multiclass_crossentropy_fixed_anchors

HUBER_DELTA = 0.5
def rcnn_smooth_l1(anchors_count,num_class):
  '''
  The shape of y_pred is (rpn_height,rpn_width,anchors_count*4*num_class)
  The shape of y_true is (rpn_height,rpn_width,anchors_count*2*4*num_class) [:,:,:anchors_count*4*num_class] is validity tensor and [:,:,anchors_count*4*num_class:] is GT
  '''
  def rcnn_smooth_l1_fixed_anchors(y_true,y_pred):

    x   = K.abs(y_true[:,:,anchors_count*4*num_class:] - y_pred)
    x   = K.switch(x < HUBER_DELTA, 0.5 * x ** 2, HUBER_DELTA * (x - 0.5 * HUBER_DELTA))
    return K.sum(y_true[:,:,:anchors_count*4*num_class]*x)/K.sum(y_true[:,:,:anchors_count*4*num_class])
  return rcnn_smooth_l1_fixed_anchors

## Train

In [0]:
base_weight_filepath = 'My Drive/SoccerAI/model/vgg16_weights.h5'
trained_weight_filepath = 'My Drive/SoccerAI/model/model_frcnn.hdf5'


In [0]:
input_test = input_tensor(input_shape)
vgg16_model = vgg16(input_test)
output = rpn(vgg16_model)
print(vgg16_model)
print(output)
output_test = K.reshape(output[1],(66600,4))
#output_test = K.squeeze(output_test,axis=0)
print(output_test)
output = ROIPoolingLayer(7)([vgg16_model,output_test])
print(output)
output = rcnn(output)
print(output)